<a href="https://colab.research.google.com/github/ericburdett/cs601r-dl/blob/master/ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Named Entity Recognition on Handwritten Documents

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, utils, datasets
from tqdm import tqdm
from torch.nn.parameter import Parameter
import pdb
import torchvision
import os
import gzip
import tarfile
from PIL import Image, ImageOps
import gc
import pdb
from IPython.core.ultratb import AutoFormattedTB
__ITB__ = AutoFormattedTB(mode = 'Verbose',color_scheme='LightBg', tb_offset = 1)

assert torch.cuda.is_available(), "Request a GPU from Runtime > Change Runtime"

In [0]:
class IamDataset(Dataset):
  def __init__(self, size=256):
    if not os.path.exists('/content/iam'):
      raise Exception('Iam dataset does not exist in /content/missionary')

    self.dataset_folder = 0

  def __getitem__(self, index):
    return self.dataset_folder[index]

  def __len__(self):
    return len(self.dataset_folder)

In [0]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, num_layers=1):
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.num_layers = num_layers

    self.cnn1 = nn.Conv2d()
    self.gru = nn.GRU(input_size, hidden_size, num_layers)
    self.linear = nn.Linear(hidden_size, output_size)

  def forward(self, input_img, hidden):
    out, hidden = self.gru(input_img, hidden)
    out = self.linear(out)

    return out, hidden

  def init_hidden(self):
    return torch.zeros(self.num_layers, 1, self.hidden_size)

In [0]:
input_size = pass
hidden_size = 200
output_size = 5
num_layers = 3

model = RNN(input_size, hidden_size, output_size, num_layers=3)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
objective = nn.CrossEntropyLoss()

dataset = IamDataset()
data_loader = DataLoader(train_dataset,
                          batch_size=1,
                          num_workers=4,
                          shuffle=False)

def train():

  for epoch in range(NUM_EPOCHS):
    loop = tqdm(total=len(data_loader), position=0, leave=True)

    for batch, (x, y_truth) in enumerate(data_loader):
      x, y_truth = x.cuda(async=True), y_truth.cuda(async=True)

      loss = 0
      hidden = model.init_hidden()
      optimizer.zero_grad()

      accs = []
      losses = []

      for word, label in zip(x, y_truth):
        optimizer.zero_grad()
        pred, hidden = model(word, hidden))

        acc = torch.eq(y_hat.argmax(), label)
        ls = objective(pred, label)

        accs.append(acc)
        losses.append(ls)

        loss += ls

      loss.backward()
      optimizer.step()

      loop.set_description('Epoch:{}, Loss:{}, Acc:{}'.format(epoch, loss, acc))
      loop.update(1)
